# 🎉 A Text Retrieval Approach Using BM25 with BERT 🎉
### TeamMGL: Manos Chatzakis, Hind El-Bouchrifi, Lluka Stojollari 

emmanouil.chatzakis@epfl.ch, hind.elbouchrifi@epfl.ch, lluka.stojollari@epfl.ch

Distributed Information Systems, EPFL, Text Retrieval project, October 2023


## Contents:
This notebook contains the following stuff:
- Imports and Libraries:
    All the imports and libraries you need to run this notebook
- Utility Functions and Objects:
    All utility functions and class definitions needed to run this notebook
- Task 1:
    Code that generates the results for Text Retrieval task
- Task 2:
    Code that generates the results for Document Reranking task
- Submission File Generation
    Code that generates the submission CSV for Kaggle

## Important Notes:

- As this notebook uses BM25, which does not represents documents as vectors, it fits the model on the fly. Please note this and exclude this time from the total running time.
    - In order to help you with time counting, we placed a tqdm counter for Task 1 to indicate exactly how much time the text retrieval requires. The expected running time for Task 1 (as we noticed by running on Kaggle) is approximately 10-12 minutes. 
    - Task 2 runs instantly
    - Although the model is fitted on the fly and it is not preloaded, fitting should not take more than a couple of minutes (4-5 minutes)
<br><br>

- This notebook is meant to run in Kaggle environment. The needed files are placed under the working directory there. Those files are:
    - Full document corpus
    - Full queries file
    - Tokenized corpus
    - Query tests sets for Task 1 and Task 2
    - Corpus Embeddings for Task 1 and Task 2
<br><br>

- This notebook contains only our final and best-scoring approach submitted in the Kaggle competition, and thus, it contains only the code needed for it. Our complete work, with all the models we implemented and used (e.g. Doc2Vec, TF-iDF, DSSM and more) as well as query expansion techniques are fully available in our GitHub repository, at: https://github.com/MChatzakis/DIS-TextRetrieval 

- You may need to install some libraries if you run this locally. 

## Imports and Libraries

Sentence transformers are used for BERT, and should be installed in the notebook environment.

In [5]:
%pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [6]:
import nltk
import math
import json
import pickle
import os
import string
import heapq
import sentence_transformers
import time
import ast
import csv

import pandas as pd
import numpy as np

from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter, defaultdict
from tqdm import tqdm
from numba import jit
from six import iteritems
from sentence_transformers import SentenceTransformer, util

# Fix below
from six.moves import range
from collections.abc import Iterable

In [7]:
# Set the global data path variable

# Path where corpus, queries and embeddings are stored
# DATA_PATH = "/kaggle/input/dis-datasets/" # Use for Kaggle
DATA_PATH = "./data/dataset/" # Use for local

# Path where test queries for Task 1 and Task 2 are stored
# DIS_PATH = "/kaggle/input/dis-project-1-text-retrieval/" # Use for Kaggle
DIS_PATH = "./data/" # Use for local

## Useful Functions and Model Definitions

### BM25 Model Definition

Here, we define the class of the bm25 Retrieval Model

In [8]:
class bm25(object):
    """bm25 class implements the BM25 ranking function."""

    def __init__(self, corpus_ids, corpus, k1=1.2, b=0.75, epsilon=0.75):
        """The consutructor for bm25 class. Loads and saves the data used for fitting.

        Args:
            corpus_ids (list): The ids of the documents in the corpus.
            corpus (list): The text corpus of the document collection.
            k1 (float, optional): k1 parameter of BM25. Defaults to 1.2.
            b (float, optional): b parameter of BM25. Defaults to 0.75.
            epsilon (float, optional): e parameter of BM25. Defaults to 0.75.
        """
        self.k1 = k1
        self.b = b
        self.epsilon = epsilon
        self.corpus_size = 0
        self.avg_doc_length = 0
        self.doc_frequencies = []
        self.idf = {}
        self.doc_lengths = []
        self.corpus = corpus
        self.corpus_ids = corpus_ids

    def fit(self):
        """
        Fits the bm25 model.
        """
        term_to_freq = defaultdict(int)
        total_length = 0

        for document in self.corpus:
            self.corpus_size += 1
            doc_length = len(document)
            total_length += doc_length
            self.doc_lengths.append(doc_length)

            frequencies = Counter(document)
            self.doc_frequencies.append(frequencies)

            for term, freq in frequencies.items():
                term_to_freq[term] += 1

        self.avg_doc_length = float(total_length) / self.corpus_size
        self.nd = term_to_freq

        idf_sum = 0
        idf_len = 0
        negative_idfs = []

        for word, freq in term_to_freq.items():
            idf = math.log((self.corpus_size + 1) / (freq))
            self.idf[word] = idf
            idf_len += 1
            idf_sum += idf
            if idf < 0:
                negative_idfs.append(word)

        self.average_idf = idf_sum / idf_len
        eps = self.epsilon * self.average_idf
        self.idf.update({word: eps for word in negative_idfs})

        document_score = {}
        for i, document in enumerate(self.corpus):
            doc_freqs = self.doc_frequencies[i]
            for word in document:
                if word not in doc_freqs:
                    continue
                score = self.idf[word] * (
                    doc_freqs[word]
                    * (self.k1 + 1)
                    / (
                        doc_freqs[word]
                        + self.k1
                        * (
                            1
                            - self.b
                            + self.b * self.doc_lengths[i] / self.avg_doc_length
                        )
                    )
                    + 1
                )

                if word not in document_score:
                    document_score[word] = {i: round(score, 2)}
                else:
                    document_score[word].update({i: round(score, 2)})
        self.document_score = document_score

    def compute_similarity(self, query, doc):
        """Computers the similarity between the query and the document.

        Args:
            query (list): Tokenized query
            doc (list): Tokenized document

        Returns:
            float: The similarity score between the query and the document.
        """
        score = 0
        doc_freqs = Counter(query)
        freq = 1
        default_idf = math.log(self.corpus_size + 1) - math.log(freq)
        for word in doc:
            if word not in doc_freqs:
                continue
            score += self.idf.get(word, default_idf) * (
                doc_freqs[word]
                * (self.k1 + 1)
                / (
                    doc_freqs[word]
                    + self.k1 * (1 - self.b + self.b * len(query) / self.avg_doc_length)
                )
                + 1
            )
        return score

    def get_top_k_documents(self, document, k=1):
        """Retrieve the top-k documents of a given document (query)

        Args:
            document (list): the document query tokens
            k (int, optional): top-k documents to be retrieved. Defaults to 1.

        Returns:
            list: a list of tuples (score, document_id, document) of the top-k documents
        """
        score_overall = {}
        for word in document:
            if word not in self.document_score:
                continue
            for key, value in self.document_score[word].items():
                score_overall[key] = score_overall.get(key, 0) + value

        k_keys_sorted = heapq.nlargest(k, score_overall, key=score_overall.get)
        return [
            (score_overall.get(item, None), self.corpus_ids[item], self.corpus[item])
            for item in k_keys_sorted
        ]

### Data Preprocessor

Here we define the Preprocessor class. The preprocessor is used for Lowercasing, Punctuation Removal, Tokenizing, Stopword Removal and Stemming.

In [9]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")  # Download WordNet data


class Preprocessor:
    """
    Preprocessor class implements the preprocessing steps for the documents and queries.
    It performs the following steps:
        1. Lowercase
        2. Remove punctuation
        3. Tokenize
        4. Remove stopwords
        5. Stemming

    Both for the documents and the queries.
    """

    def __init__(self):
        """Initialize the basic preprocessing structures."""
        self.stemmer = PorterStemmer()
        self.stopwords = set(stopwords.words("english"))
        self.punctuation = set(string.punctuation)

    def preprocess(self, documents):
        """Preprocess a collection of documents.

        Args:
            documents (list or dict): list or dict{id: document} of documents to be preprocessed

        Raises:
            TypeError: In case the documents are not a list or a dictionary

        Returns:
            list or dict: The tokenized documents
        """
        tokenized_docs = []
        if isinstance(documents, list):
            tokenized_docs = self.preprocess_document_list(documents)
        elif isinstance(documents, dict):
            tokenized_docs = self.preprocess_document_dict(documents)
        else:
            raise TypeError("Documents must be either a list or a dictionary")

        return tokenized_docs

    def preprocess_query(self, query):
        """Preprocess a query

        Args:
            query (str): The query to be processed

        Returns:
            list: Preprocessed query terms
        """
        query = self.tolowercase(query)
        query = self.remove_punctuation(query)

        query_tokens = self.tokenize(query)
        query_tokens = self.remove_stopwords(query_tokens)

        query_tokens = self.stem(query_tokens)

        return query_tokens

    def preprocess_document_list(self, document_list):
        """Preprocess a list of documents

        Args:
            document_list (_type_): _description_

        Returns:
            list: list of tokenized documents
        """
        tokenized_docs = []
        for i in range(len(document_list)):
            tokenized_docs.append(self.preprocess_doc(document_list[i]))
        return tokenized_docs

    def preprocess_document_dict(self, document_dict):
        """Preprocess a dictionary of documents

        Args:
            document_dict (dict): Dictionary of documents {id: document}

        Returns:
            dict: Dictionary of tokenized documents {id: tokenized_document}
        """
        tokenized_docs = {}
        for doc_id in document_dict.keys():
            document = document_dict[doc_id]
            tokenized_docs[doc_id] = self.preprocess_doc(document)
        return tokenized_docs

    def preprocess_doc(self, document):
        """Basic preprocessing pipeline for a document

        Args:
            document (str): _description_

        Returns:
            list: preprocess document tokens
        """
        document = self.tolowercase(document)
        document = self.remove_punctuation(document)

        document_tokens = self.tokenize(document)
        document_tokens = self.remove_stopwords(document_tokens)
        document_tokens = self.stem(document_tokens)

        return document_tokens

    def tolowercase(self, document):
        return document.lower()

    def remove_punctuation(self, document):
        return "".join([char for char in document if char not in self.punctuation])

    def tokenize(self, document):
        return word_tokenize(document)

    def remove_stopwords(self, tokens):
        return [token for token in tokens if token not in self.stopwords]

    def stem(self, tokens):
        return [self.stemmer.stem(token) for token in tokens]

    def save_docs(self, docs, path):
        with open(path, "w") as jsonl_file:
            for docID in docs:
                doc_data = {"_id": str(docID), "tokens": docs[docID]}
                json_line = json.dumps(doc_data)
                jsonl_file.write(json_line + "\n")

    def load_docs(self, path):
        raw_queries = {}
        with open(path, "r") as file:
            for line in file:
                data = json.loads(line)
                raw_queries[data["_id"]] = data["tokens"]

        return raw_queries

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/manoschatzakis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/manoschatzakis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/manoschatzakis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Utility Functions

Here, we define all the functions needed for various tasks of our pipeline.

In [10]:
def cosine_similarity(vectors, query):
    """
    Calculates cosine similarity between two vectors (or matrix and vector)

    Args:
        vectors: X : Matrix X (np.array of vectors)
        query:   y : Vector y (np.array of query vector)

    Returns:
        number: cosine similarity between X and y
    """
    dot_products = np.dot(vectors, query)

    norm_target = np.linalg.norm(query)
    norm_vectors = np.linalg.norm(vectors, axis=1)

    # Calculate the cosine similarity between the target vector and all vectors in the array
    return dot_products / (norm_target * norm_vectors)


@jit(nopython=True)
def cosine_similarity_nb(u: np.ndarray, v: np.ndarray):
    """An optimized version of cosine similarity.

    Args:
        u (np.ndarray): u
        v (np.ndarray): v

    Returns:
        float: The cosine similarity between u and v
    """
    assert u.shape[0] == v.shape[0]
    uv = 0
    uu = 0
    vv = 0
    for i in range(u.shape[0]):
        uv += u[i] * v[i]
        uu += u[i] * u[i]
        vv += v[i] * v[i]
    cos_theta = 1
    if uu != 0 and vv != 0:
        cos_theta = uv / np.sqrt(uu * vv)
    return cos_theta


@jit(nopython=True)
def cosine_similarity_nb_matrix(X: np.ndarray, y: np.ndarray):
    """An optimized version of cosine similarity, supporting matrices.

    Args:
        X (np.ndarray): X
        y (np.ndarray): y

    Returns:
        float: The cosine similarity for each vector X[i]*y
    """
    cos_theta = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        cos_theta[i] = cosine_similarity_nb(X[i], y)
    return cos_theta


def load_jsonl_data(data_path: str, key_name: str, value_name: str):
    """Load data from a JSONL file and return IDs, texts, and a dictionary mapping ids to text."""
    ids, texts, dict_ = [], [], {}

    with open(data_path, "r") as file:
        for line in file:
            data = json.loads(line)
            ids.append(data[key_name])
            texts.append(data[value_name])
            dict_[data[key_name]] = data[value_name]

    return ids, texts, dict_

## Task 1: Text Retrieval

In Task 1 we are given a set of approximately 7.5K queries and we retrieve the relevant documents over a corpus of 1.5M documents. 

Our retrieval method is based on the following method:
* Retrieve the top-N relevant documents of a query from the corpus, with N in {20,30,50,100}
* Rerank the retrieved top-N documents to get the final top-10 results using BERT

### Step 1: Load the corpus, queries and preprocessed corpus

Everything is stored on Kaggle workspace, accessible from this notebook

In [11]:
# The preprocessed corpus is preprocessed using the Preprocessor class once, and we have stored the results locally.
RAW_CORPUS = f"{DATA_PATH}corpus.jsonl"
QUERIES = f"{DATA_PATH}queries.jsonl"
TOKENIZED_CORPUS = f"{DATA_PATH}tokenized_corpus.jsonl"

# We specify the names of the fields in the JSONL files that we want to load.
document_ids, documents, docs_dict = load_jsonl_data(RAW_CORPUS, "_id", "text")

query_ids, queries, queries_dict = load_jsonl_data(QUERIES, "_id", "text")

tokenized_document_ids, tokenized_documents, tok_docs_dict = load_jsonl_data(
    TOKENIZED_CORPUS, "_id", "tokens"
)

print(f"Number of documents: {len(documents)}")
print(f"Number of queries: {len(queries)}")

Number of documents: 1471406
Number of queries: 509962


### Step 2: Load the precomputed corpus embeddings.

We have precomputed the corpus embeddings in order to boost the performance during query answering

In [12]:
# Load the object from the pickle file
CORPUS_EMB_PATH = f"{DATA_PATH}embeddings_final.pkl"
with open(CORPUS_EMB_PATH, "rb") as file:
    corpus_emb = pickle.load(file)

EMBEDDING_DIM = list(corpus_emb.values())[0].shape[0]
print(f"Dimensions of BERT embeddings: {EMBEDDING_DIM}")

Dimensions of BERT embeddings: 384


### Step 3: Load the test query set for Task 1.

The models are evaluated on Kaggle using a query set of approximately 7.5K queries for Task 1.

In [13]:
# Load the task 1 test queries
TEST_QUERIES_PATH_T1 = f"{DIS_PATH}task1_test.tsv"
t1_test_queries_df = pd.read_csv(TEST_QUERIES_PATH_T1, delimiter="\t")

print(f"Number of test queries for Task 1: {len(t1_test_queries_df)}")

Number of test queries for Task 1: 7437


### Step 4: Fit the BM25 Model.

Given the nature of the BM25 Model, we are unable to efficiently save the model and reuse it. Thus we fit it from scratch here. Locally fit takes just 1 minute! On Kaggle, it should need 3-4 minutes.

In [14]:
# Instanciate and fit the BM25 model and count time
st = time.time()

model = bm25(tokenized_document_ids, tokenized_documents)
model.fit()

et = time.time()

print(f"BM25 fitted. Elapsed time: {(et - st)} seconds")

BM25 fitted. Elapsed time: 59.982434034347534 seconds


### Step 5: Retrieve the relevant documents!

Now is the time to start retrieving the relevant documents for each one of the given queries! To do this, we split the computations into two parts for each query:
    a. Use BM25 to retrieve the top-n relevant documents for each query
    b. Use BERT Sentence Embeddings to rerank the retrieved set and obtain the top-10 documents for recall@10!

In [15]:
# Select top N documents to retrieve per query
TOP_N = 100

# Instanciate the preprocessor
preprocessor = Preprocessor()

# Iterate over the queries and keep the IDs of the top 100 documents
top_N = []

# Instanciate the SentenceTransformer model
st_bert = SentenceTransformer("all-MiniLM-L12-v2")

# List to store the results (we need them at the end to generate the submission file)
data = []

# Set the number of documents to retrieve per query (Top-10 for Recall@10)
k = 10

for idx, row in tqdm(t1_test_queries_df.iterrows()):
    #
    # Part 1: Retrieve the top-N relevant documents using BM25
    #

    # Load query data
    id = row["id"]
    query_id = row["query-id"]
    query = queries_dict[str(query_id)]

    # Preprocess the query and generate its terms
    tokenized_query = preprocessor.preprocess_query(query)

    # Get the resulting documents. The format is: (score, document_id, document_terms)
    result = model.get_top_k_documents(tokenized_query, k=TOP_N)

    #
    # Part 2: Rerank and select top-10 using BERT
    #

    # Get the embeddings of the query
    query_embedding = st_bert.encode(query, show_progress_bar=False)

    # Calculate the cosine similarity between the query embeddings and each document embedding
    cos_similarities = []
    index_to_docID = {}
    for i, r in enumerate(result):
        c_id = int(r[1])
        index_to_docID[i] = c_id
        cos_similarities.append(
            cosine_similarity_nb(corpus_emb[c_id], query_embedding)
        )

    # Sometimes, BM25 might not retrieve enough documents, so we need to check
    curr_k = k
    if len(cos_similarities) < k:
        curr_k = len(cos_similarities)

    # Get the indices of the top 'topn' most similar documents
    top_indices = np.argpartition(cos_similarities, -curr_k)[-curr_k:]

    # Getdocument indices of the top documents
    top_k_docIDs = [index_to_docID[index] for index in top_indices]

    # Append the data to the list
    data.append((idx, top_k_docIDs, -1))


print("Task 1 Done!")

7437it [04:17, 28.92it/s]

Task 1 Done!


### That was it for Task 1! We kept the results in the "data" list, in order to generate the submission file in the end! ✅

## Task 2: Document Re-ranking

For the document reranking task, we use BERT embeddings to recalculate the score of the given relevant documents.

### Step 1: Load the Task 2 test queries

In [16]:
T2_TEST_QUERIES_PATH = f"{DIS_PATH}task2_test.tsv"
t2_test_queries_df = pd.read_csv(T2_TEST_QUERIES_PATH, delimiter="\t")

print(f"Number of test queries for Task 2: {len(t2_test_queries_df)}")

Number of test queries for Task 2: 33


### Step 2: Load the embeddings corresponding to Task 2

In [17]:
# Load the Task 2 embeddings from the precomputed pickle file
CORPUS_EMB_PATH = f"{DATA_PATH}embeddings_task2.pkl"
with open(CORPUS_EMB_PATH, "rb") as file:
    corpus_emb_t2 = pickle.load(file)

EMBEDDING_DIM = list(corpus_emb_t2.values())[0].shape[0]
print(f"Dimension of BERT embeddings: {EMBEDDING_DIM}")

Dimension of BERT embeddings: 384


### Step 3: Rerank the given documents using BERT embeddings

In [19]:
# In order to not lose time, we resuse the SentenceTransformer model from Task 1
# st_bert = SentenceTransformer("all-MiniLM-L12-v2")

# For easiness, we reuse the data list from Task 1
# data = []

# This task is relatively fast
for _, row in t2_test_queries_df.iterrows():
    # Get the query data
    id = row["id"]
    query_id = row["query-id"]
    query = queries_dict[str(query_id)]

    # Infer the embedding vector for this query (this is performed at runtime, on the fly)
    query_embedding = st_bert.encode(query, show_progress_bar=False)

    # Get the corpus IDs for this query (ast is used to convert the string to a list)
    corpus_ids = ast.literal_eval(row["corpus-id"])

    # Generate the scores for each one of the retrieved documents
    scores = []
    for corpus_id in corpus_ids:
        doc_emb = corpus_emb_t2[corpus_id]

        # Add 2 to the cosine similarity score to make it positive (scaling up)
        score = cosine_similarity_nb(doc_emb, query_embedding) + 2
        scores.append(score)

    # Reusing the data list from Task 1
    data.append((id, -1, scores))

## Submission File Generation

Here, we generate the final submission file for both tasks. The file is saved in Kaggle workspace.

In [20]:
# Save the results stored in data list to a CSV file
csv_file = "output.csv"

with open(csv_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["id", "corpus-id", "score"])
    writer.writerows(data) # Data holds the results from Task 1 and Task 2

print(f"CSV file '{csv_file}' has been created.")

CSV file 'output.csv' has been created.


## The End! 🎉

If everything went fine, the query answering phase to generate the CSV file will score approximately 75% on Kaggle competition. Locally (using MacOS with m1-pro) we are able to generate the csv within 8 minutes. In Kaggle, it takes a bit more, approximately 10-12 minutes. 👩‍💻